EXERCISE 1 - Use python datetime library to measure how long it took to 
send the data. To do so you need to import it

In [1]:
import socket
import datetime
import time 
# i am using time for a more accurate reading

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))

message = input("type here-->")

start_time = time.perf_counter()  

client_socket.sendall(message.encode())

response = client_socket.recv(1024)

end_time = time.perf_counter()  

elapsed_time = (end_time - start_time )* 1e6

print(f"Server response: {response.decode()}")
print(f"time taken: {elapsed_time:.3f} microseconds")

client_socket.close()


Server response: ACK: file_to_send.txt
time taken: 372.800 microseconds


Server response: ACK: file_to_send.txt<br>
time taken: 372.800 microseconds<br>

EXERCISE 2: Change your script so that the data will be send using UDP 
and compare the time needed to send it using both protocols. 

In [4]:
import socket
import time
SERVER_HOST = 'localhost'
UDP_PORT = 65433

message = input("type here-->")

client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

start_time = time.perf_counter()  

client_socket.sendto(message.encode(), ('localhost', 65433))

response, _ = client_socket.recvfrom(1024)

end_time = time.perf_counter()  

elapsed_time = (end_time - start_time ) * 1e6


print(f"UDP Server response: {response.decode()}")
print(f"time taken to send and receive response: {elapsed_time:.3f} microseconds")

client_socket.close()


UDP Server response: ACK: hellooooooooo
time taken to send and receive response: 901.900 microseconds


UDP Server response: ACK: hellooooooooo<br>
time taken to send and receive response: 901.900 microseconds<br>

EXERCISE 3 - sending text files and comparing the time they are sent based off of size of file<br>
Small file - Contained my name<br>
Large file - Contained the first chapters of Frankenstein<br>

In [ ]:
import socket
import datetime
import time

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))

file_name = "file_to_send.txt"  

try:
    with open(file_name, 'rb') as f:
        print(f"sending file '{file_name}' to server...")

        file_start_time = time.perf_counter()  

        while (chunk := f.read(1024)): 
            client_socket.sendall(chunk)

    client_socket.shutdown(socket.SHUT_WR)

    response = client_socket.recv(1024)

    file_end_time = time.perf_counter()  
    file_transfer_time = (file_end_time - file_start_time) * 1e6

    print(f"server response: {response.decode()}")
    print(f"time taken to send the file: {file_transfer_time:.3f}  microseconds")

except FileNotFoundError:
    print(f"File '{file_name}' not found.")

client_socket.close()
print("Connection closed.")



for the small.txt = 468.600 microseconds
for large.txt = 2661.200 microseconds

EXERCISE 4 - TXT FILE TRANSFER USING UDP

In [ ]:
import socket
import time

SERVER_HOST = 'localhost'
UDP_PORT = 65432
file_name = "file_to_send.txt"  
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

try:
    print(f"sending file '{file_name}' to server...")

    client_socket.sendto(file_name.encode(), (SERVER_HOST, UDP_PORT))

    file_start_time = time.perf_counter()

    with open(file_name, 'rb') as f:
        while (chunk := f.read(1024)): 
            client_socket.sendto(chunk, (SERVER_HOST, UDP_PORT))

    client_socket.sendto(b"EOF", (SERVER_HOST, UDP_PORT))

    response, _ = client_socket.recvfrom(1024)

    file_end_time = time.perf_counter()
    file_transfer_time = (file_end_time - file_start_time) * 1e6 

    print(f"server response: {response.decode()}")
    print(f"time taken to send the file: {file_transfer_time:.3f} microseconds")

except FileNotFoundError:
    print(f"File '{file_name}' not found.")

client_socket.close()
print("UDP Connection closed.")


sending file 'file_to_send.txt' to server...


Time taken to send the  large.txt file using UDP: 67590.700 microseconds<br>
Time taken to send the small.txt file using UDP : 2222.000 microseconds<br>



EXCERISE 1,2,3,4

In [ ]:
import socket
import threading
import datetime

def client_task(client_id, mode, filename=None):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('localhost', 65432))

    greeting = client_socket.recv(1024).decode()
    print(f"[{datetime.datetime.now()}] client {client_id} received: {greeting}")

    if mode == "CHAT":
        client_socket.sendall(b"CHAT")

        message = f"client {client_id}: wassup server"
        print(f"[{datetime.datetime.now()}] client {client_id} sending message: {message}")

        client_socket.sendall(message.encode())

        response = client_socket.recv(1024)
        print(f"[{datetime.datetime.now()}] client {client_id} received: {response.decode()}")

    elif mode == "FILE":
        client_socket.sendall(b"FILE")

        if filename:
            try:
                client_socket.sendall(filename.encode())

                with open(filename, 'rb') as f:
                    while (chunk := f.read(1024)):
                        client_socket.sendall(chunk)

                print(f"[{datetime.datetime.now()}] client {client_id} sent file '{filename}'")

                response = client_socket.recv(1024)
                print(f"[{datetime.datetime.now()}] client {client_id} received: {response.decode()}")

            except FileNotFoundError:
                print(f"Client {client_id}: not found")

    client_socket.close()

num_clients = 2
option = input(" 1 to send messages, 2 for send a file: ")

client_threads = []

if option == "1":
    for i in range(num_clients):
        thread = threading.Thread(target=client_task, args=(i, "CHAT"))
        thread.start()
        client_threads.append(thread)

elif option == "2":
    filename = input("filename: ")

    for i in range(num_clients):
        thread = threading.Thread(target=client_task, args=(i, "FILE", filename))
        thread.start()
        client_threads.append(thread)
for thread in client_threads:
    thread.join()



EXERCISE 5 - Multiple Clients that want to talk

paste the client code into a new window + press empty space bar to load pending chats

In [1]:

import socket
import threading

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))

print("Connected to chat server! Type messages below (type 'exit' to quit).")

def receive_messages():
    while True:
        try:
            message = client_socket.recv(1024).decode()
            if not message:
                break
            print("\n" + message)  
        except ConnectionResetError:
            print("Server disconnected")
            break

receive_thread = threading.Thread(target=receive_messages)
receive_thread.start()

while True:
    msg = input()
    if msg.lower() == "exit":
        break
    client_socket.sendall(msg.encode())

client_socket.close()

Connected to chat server! Type messages below (type 'exit' to quit).

Client 1: heyyoo

Client 2: hey im the 3rd client

Client 2: heyy

Client 1: heyo


Exception in thread Thread-3 (receive_messages):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\Alexis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure


    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Alexis\AppData\Local\Temp\ipykernel_21036\2950043864.py", line 12, in receive_messages
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine


![title](Images/Results_8.png)

EXERCISE 6 - tcp client for only two client that want to talk, ensure code is opened in seperate window

In [ ]:

import socket
import threading

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))

print("Connected to chat server! Type messages below (type 'exit' to quit).")


def receive_messages():
    while True:
        try:
            message = client_socket.recv(1024).decode()
            if not message:
                break
            print("\n" + message)  
        except ConnectionResetError:
            print("Server disconnected")
            break

receive_thread = threading.Thread(target=receive_messages)
receive_thread.start()

while True:
    msg = input()
    if msg.lower() == "exit":
        break
    client_socket.sendall(msg.encode())

client_socket.close()



EXERCISE 7 AND 8 

In [ ]:
import socket
import requests
from cryptography.fernet import Fernet
api_url = "https://api.open-meteo.com/v1/forecast?latitude=51.47&longitude=-0.0363&current_weather=true"

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))

key_length = int.from_bytes(client_socket.recv(4), 'big') 
key = client_socket.recv(key_length)
cipher = Fernet(key)

message = "hey server"
response = requests.get(api_url)
weather_data = response.json()
weather_output = f"temp:{weather_data['current_weather']['temperature']},Wind Speed: {weather_data['current_weather']['windspeed']} km/h"
print(f"Fetched Weather: {weather_output}")

entire_message = f"{message} | {weather_output}"

encrypted_message = cipher.encrypt(entire_message.encode())
print(f"Encrypted Message Sent: {encrypted_message}")

client_socket.sendall(encrypted_message)

encrypted_response = client_socket.recv(1024)
decrypted_response = cipher.decrypt(encrypted_response).decode()

print(f"Server decrypted: {decrypted_response}")

client_socket.close()
